In [ ]:
import pickle
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time
from retrieval_models.reranking import Reranker

C:\Users\NAVHI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Replace 'path_to_your_file.pkl' with the actual path to your PKL file
file_path = "C:/Users/NAVHI/Downloads/25queries_results.pkl"

# Open the file in binary mode and load the data
with open(file_path, 'rb') as file:
    data = pickle.load(file)

# Now 'data' contains the deserialized Python object
print(data["TypeError: 'NoneType' object is not subscriptable in Python"])
print(data.keys())

['1490039', '1308079', '631788', '1689012', '1449620', '1689145', '768941', '1115313', '1772491', '550585', '550915', '768956', '1529527', '1689139', '999182', '1728697', '550868', '1223197', '1601656', '506956', '1646326', '1264818', '1385753', '772220', '745600', '1520897', '1593100', '788935', '550829', '598776', '1690080', '951974', '646652', '1442686', '1382252', '550474', '784584', '1036512', '930183', '1633691', '1548620', '1442675', '1359557', '615129', '885158', '898326', '533768', '717655', '817937', '1308469']
dict_keys(["TypeError: 'NoneType' object is not subscriptable in Python", 'NullPointerException handling in Java', "ModuleNotFoundError: No module named 'requests'", 'Segmentation fault in C++"', "SyntaxError: invalid syntax near 'elif'", 'Difference between DFS and BFS algorithms', 'How does garbage collection work in Java?', 'What is tail recursion, and how does it optimize memory', 'What are strong and weak references in Python?', 'Explain dynamic programming with a

In [3]:
documents = {}  

with open("C:/Users/NAVHI/Downloads/test_documents (1).txt", "r", encoding="utf-8") as file:  
    for line in file:  
        parts = line.split("\t", maxsplit=1)
        if len(parts) == 2:  
            doc_id = parts[0].strip()  
            text = parts[1].strip()  
            documents[doc_id] = text  


In [4]:
def LM(document):
   model_name = "t5-small"  # You can change this based on the model you're using
   tokenizer = AutoTokenizer.from_pretrained(model_name)
   model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

   # Preprocess Input for Query Generation
   input_text = "generate query: " + document
   input_ids = tokenizer.encode(input_text, return_tensors="pt")

   # Generate Query
   output_ids = model.generate(input_ids, max_length=50)
   generated_query = tokenizer.decode(output_ids[0], skip_special_tokens=True)

   return generated_query

In [5]:
doc_dict={}

start_time = time.time()

for key, value in data.items():
    
    # Ensure key exists in doc_dict before appending
    if key not in doc_dict:
        doc_dict[key] = {}

    for j in range(len(value)):
        
        doc_id = value[j]
        if doc_id in documents:
            doc = documents[doc_id]
            
            generated_query = LM(doc)
            
            # Ensure the generated query is not None
            if generated_query:
                doc_dict[key][doc_id]=generated_query
            else:
                print(f"Warning: LM returned None for document ID {doc_id}")
        else:
            print(f'Warning: Document ID {doc_id} not found in documents dictionary')
# Example output for a specific key
print(doc_dict.get("How to make an API request with Axios in JavaScript?", "Key not found"))

end_time = time.time()
# **Calculate Execution Time**
execution_time = end_time - start_time

with open("generated_query.txt", "w", encoding="utf-8") as f:
    for key, value in doc_dict.items():
        f.write(key)  # Writing the question (dictionary key)
        f.write(":\n")  # Colon separator and new line

        for keys,values in value.items():
            f.write(keys)  # Writing the generated query
            f.write("\n")
            f.write(values)
            f.write("\n")# New line for each query

        f.write("\n")  # Extra newline for readability between entries

print(f"Execution Time: {execution_time:.6f} seconds")


Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


{'1547201': 'ans*ans = x', '1547196': 'def sqrt(x)', '1724304': 'dbo.funnyInsert', '1724137': 'ExSheet/ExSubject', '1516174': 'RadioButtonList', '1547210': 'def sqrt(x)', '1285655': 'I', '1529201': 'stdio.h>', '1289174': 'submitAnswer()', '1289099': 'submitAnswer()', '1547224': 'x>1', '562770': 'FROM', '829525': 'Jon Limjap', '1289135': 'input type="radio" id="ans1" name="ans1" value="ans" value="1" />', '1736340': 'UDF create function dbo.ExtractName(@TheName VARCHAR(200),@Which CHAR(1) )', '1737786': 'DECLARE @ans varchar(1);', '1288985': 'value="1"', '1751048': 'JAXB', '1737763': 'case', '1288904': 'ajax', '1697816': 'def flatten((()', '1192007': 'Listint>()', '654010': 'tried', '1543722': 'C', '1289139': 'Try: var ansVal = myForm.ans.checkd', '1723877': 'questions', '608284': '$ans', '674477': 'isempty', '712325': 'ansSeries', '450437': 'try :', '716481': 'Google', '1289152': 'submitAnswer()', '1530487': 'current_ans', '1508979': 'WinForm application', '578567': 'b = 2', '1407076':

In [6]:
def similarity_score(user_query,generated_query):
    from sentence_transformers import SentenceTransformer, util

    # Load Pretrained SBERT Model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    # Encode both queries
    user_embedding = model.encode(user_query, convert_to_tensor=True)
    generated_embedding = model.encode(generated_query, convert_to_tensor=True)

    # Compute Cosine Similarity
    similarity_score = util.pytorch_cos_sim(user_embedding, generated_embedding).item()

    return similarity_score



In [7]:
start_time = time.time()
reranked_as_per_LM_doc={}
for key, value in doc_dict.items():
    score_doc = {}  # Reset for each key
    for keys, values in value.items():
        score = similarity_score(key, values)
        score_doc[keys] = score
    
    # Sort documents based on similarity score (ascending order)
    sorted_docs = sorted(score_doc.items(), key=lambda x: x[1])
    reranked_as_per_LM_doc[key] = sorted_docs

end_time = time.time()

# Calculate Execution Time
execution_time = end_time - start_time

print(reranked_as_per_LM_doc)

    

        
        
        
        

{"TypeError: 'NoneType' object is not subscriptable in Python": [('1601656', -0.1768856644630432), ('745600', -0.09816920012235641), ('768956', -0.09536277502775192), ('1264818', -0.07530263066291809), ('615129', -0.06032320857048035), ('1728697', -0.04657905548810959), ('1382252', -0.043050240725278854), ('717655', -0.039162926375865936), ('550585', -0.03867025673389435), ('550915', -0.03867025673389435), ('550868', -0.03867025673389435), ('598776', -0.03867025673389435), ('550474', -0.03867025673389435), ('1359557', -0.03867025673389435), ('533768', -0.03547365218400955), ('1442686', -0.027986520901322365), ('885158', -0.015722468495368958), ('898326', -0.014893196523189545), ('1308079', -0.012226486578583717), ('951974', -0.009180951863527298), ('1593100', 0.0006808619946241379), ('930183', 0.024030257016420364), ('646652', 0.03650932013988495), ('1036512', 0.037919238209724426), ('1442675', 0.03886443376541138), ('817937', 0.0659407377243042), ('506956', 0.08202903717756271), ('168